IWDGD中大值的确定性噪声 对应到DGD算法中是固定的对偶攻击值为-24 这样两个算法各自迭代2000次 所有用户的对偶迭代值均在-23.83附近

鲁棒聚合器IOS 

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(54,2000)) 
dual_variable = np.zeros(shape=(54,2000))#对偶变量
sum_resource=np.zeros(shape=(1,2000), dtype=np.float64)
sum_cost=np.zeros(shape=(1,2000), dtype=np.float64)
iteration_update = 0.01 #迭代步长
#W=np.zeros(shape=(54,54)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
W=np.loadtxt("double_stochastic_weight_1.txt") #双随机的网络拓扑矩阵W（对角线处非零）
iteration=[]#绘图横坐标
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
A=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑邻接矩阵（对角线处为0）
D=np.loadtxt("degree_generator.txt") #用户的度
neighbor=np.zeros(shape=(54,17))#定义矩阵存放54个用户的邻居
neighbor_help=[]#辅助求用户邻居的列表
q=1

In [3]:
#对偶变量的初始化 初始化一致
for i in range (0,54):
    dual_variable[i][0]=0
    
#资源量的初始化 资源量的初始化需要和对偶变量的初始值满足某种关系
for i in range (0,54):
    agent_resource_quantity[i][0]=6000/54

iteration.append(0)

#总资源量 初始化
sum=0
for i in range (0,54):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

#用户总成本初始化
sum=0
for i in range(0,54):
    sum=sum+(generator_c_values[i]*(agent_resource_quantity[i][0])*(agent_resource_quantity[i][0])+generator_b_values[i]*agent_resource_quantity[i][0]+generator_a_values[i])
sum_cost[0][0]=sum

#对用户邻居进行初始化
for i in range(0,54):
    for j in range(0,17):
        neighbor[i][j]=99 #先将矩阵中的所有数字设定为一个不可能的数字
#根据邻接矩阵求每个用户的邻居集合
for i in range(0,54):
    for j in range(0,54):
        if A[i][j]!=0:
            neighbor_help.append(j)
l=0
for i in range(0,54):
    g=0
    for j in range(l,l+int(D[i])):
        neighbor[i][g]=neighbor_help[j]  
        g=g+1
    l=l+int(D[i])    

In [4]:
#DGD算法 
def DGD_IOS_determinitic_bias(D,neighbor,generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,dual_variable,iteration_update,W,iteration):
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+无计算误差****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        iteration.append(k)
        
        #所有用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate=[] #存放当前迭代中54个用户对对偶变量做完梯度下降后得到的中间值
        for i in range(0,54):
            #每个用户对对偶变量先进行一次梯度下降
            if i==48:
                dual_variable_intermediate.append(-24)
            else:
                dual_variable_intermediate.append(dual_variable[i][k-1]-iteration_update*((6000/54)-agent_resource_quantity[i][k-1]))             
        #阶段二：鲁棒聚合 IOS       
        for i in range(0,54):
            #先根据用户的度来判断其是否需要做IOS操作 
            if int(D[i])>=2: 
            #if i==44 or i==47 or i==48:
                u_neighbor_help=[]
                for j in range(0,int(D[i])):
                    u_neighbor_help.append(int(neighbor[i][j])) 
                u_neighbor_help.append(i)
                for q_help in range(0, int(q)):#剔除q个邻居的信息
                    dual_variable_weight_average=0
                    h=0
                    for j in range(0,len(u_neighbor_help)):#计算当前循环中的h值
                        h=h+W[i][u_neighbor_help[j]]
                    for j in range(0,len(u_neighbor_help)):#计算当前循环中的对偶权重平均值
                        dual_variable_weight_average=dual_variable_weight_average+(1/h)*(W[i][u_neighbor_help[j]]*dual_variable_intermediate[u_neighbor_help[j]])
                    #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                    neighbor_index=u_neighbor_help[0]
                    max_distance=abs(dual_variable_intermediate[u_neighbor_help[0]]-dual_variable_weight_average)
                    for j in range(1,len(u_neighbor_help)):
                        if abs(dual_variable_intermediate[u_neighbor_help[j]]-dual_variable_weight_average)>=max_distance:
                            max_distance=abs(dual_variable_intermediate[u_neighbor_help[j]]-dual_variable_weight_average)
                            neighbor_index=u_neighbor_help[j]
                    u_neighbor_help.remove(neighbor_index)
                dual_variable_weight_average=0
                h=0  
                for j in range(0,len(u_neighbor_help)):
                    h=h+W[i][u_neighbor_help[j]]
                for j in range(0,len(u_neighbor_help)):
                    dual_variable_weight_average=dual_variable_weight_average+(1/h)*(W[i][u_neighbor_help[j]]*dual_variable_intermediate[u_neighbor_help[j]]) 
                dual_variable[i][k]=dual_variable_weight_average
            else:
                mix_sum_dual=0           
                for j in range(0,54):
                    mix_sum_dual=mix_sum_dual+W[i][j]*dual_variable_intermediate[j]
                dual_variable[i][k]=mix_sum_dual                    
        
        #用户资源申请量的迭代
        for i in range (0,54):
            agent_resource_quantity[i][k]=(-dual_variable[i][k]-generator_b_values[i])/(2*generator_c_values[i]) 

        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,54):
            sum2=sum2+(generator_a_values[i]+(generator_b_values[i]*agent_resource_quantity[i][k])+(generator_c_values[i]*agent_resource_quantity[i][k]*agent_resource_quantity[i][k]))
        sum_cost[0][k]=sum2
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%20==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,54):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            for i in range(0,54):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=(1/(pow(k+1,1.3)))*(1/26.511) #递减步长
        iteration_update=(1/(pow(k+1,1.5)))*(1/18.8468) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

np.savetxt('../data/01-DGD-IOS-robust-agent_resource_quantity_large_value_deterministic_bias.txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/01-DGD-IOS-robust-sum_cost_large_value_deterministic_bias.txt',sum_cost,fmt='%0.8f')#用户成本总和
np.savetxt('../data/01-DGD-IOS-robust-sum_resource_large_value_deterministic_bias.txt',sum_resource,fmt='%0.8f')#用户资源总和

In [5]:
DGD_IOS_determinitic_bias(D,neighbor,generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,dual_variable,iteration_update,W,iteration)

****************************DGD算法+无计算误差****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 111.11111111111111
agent 2 的初始资源申请量为: 111.11111111111111
agent 3 的初始资源申请量为: 111.11111111111111
agent 4 的初始资源申请量为: 111.11111111111111
agent 5 的初始资源申请量为: 111.11111111111111
agent 6 的初始资源申请量为: 111.11111111111111
agent 7 的初始资源申请量为: 111.11111111111111
agent 8 的初始资源申请量为: 111.11111111111111
agent 9 的初始资源申请量为: 111.11111111111111
agent 10 的初始资源申请量为: 111.11111111111111
agent 11 的初始资源申请量为: 111.11111111111111
agent 12 的初始资源申请量为: 111.11111111111111
agent 13 的初始资源申请量为: 111.11111111111111
agent 14 的初始资源申请量为: 111.11111111111111
agent 15 的初始资源申请量为: 111.11111111111111
agent 16 的初始资源申请量为: 111.11111111111111
agent 17 的初始资源申请量为: 111.11111111111111
agent 18 的初始资源申请量为: 111.11111111111111
agent 19 的初始资源申请量为: 111.11111111111111
agent 20 的初始资源申请量为: 111.11111111111111
agent 21 的初始资源申请量为: 111.11111111111111
agent 22 的初始资源申请量为: 111.11111111111111
agent 23 的初始资源申请量为: 111.11111111111111
agent 24 的初始资源申请量为: 111.1111111111

In [6]:
np.savetxt('../data/01-DGD-IOS-robust-agent_resource_quantity_large_value_deterministic_bias.txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/01-DGD-IOS-robust-sum_cost_large_value_deterministic_bias.txt',sum_cost,fmt='%0.8f')#用户成本总和
np.savetxt('../data/01-DGD-IOS-robust-sum_resource_large_value_deterministic_bias.txt',sum_resource,fmt='%0.8f')#用户资源总和